### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221202_scenario_projections.ipynb")

2022/12/01 11:12:24 INFO|============================================================


### Fit an initial scenario through present day

In [50]:
# set inputs for the model

# first establish the model start date
startdate = dt.date(2020, 1, 24).isoformat()
# then get today's date (so we can set end dates automatically)
today = dt.datetime.today().strftime("%Y-%m-%d")
# now calculate the end date (let's do 90 days from today's date, but can be as long we we want)
plus90 = dt.datetime.strptime(today, "%Y-%m-%d").date() + dt.timedelta(days = 90)
enddate = plus90.isoformat()

# set model arguments
model_args = {
    'params_defs': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_scenario_params.json',
    #'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_vacc_proj_params.json',
    'start_date': startdate,
    'end_date': enddate
}

# set output directory and end date for fit (which is today's date)
fit_args = {
    'outdir': outdir,
    'fit_end_date': today
}

In [ ]:
# if you want to refit the model, run this code
# this creates an object called "model" by passing the fit_args and model_args through the do_single_fit function
# otherwise, run the next cell down
#model = do_single_fit(**fit_args, **model_args)

In [51]:
# if you want to pass an old fit spec ID through, run this code
# otherwise, refit using the code above
model_args = {
    'base_spec_id': 4515, # use the spec id that was output from the model fit you want to pass through
}
# create an object called "model" by passing the above-mentioned model_args through CovidModel
model = CovidModel(**model_args)
# prep the model
model.prep()

2022/12/01 11:39:31 INFO|------------------------|{'fit_batch': '34', 'run_type': 'fit'} Retrieving vaccinations data
2022/12/01 11:39:34 INFO|------------------------|{'fit_batch': '34', 'run_type': 'fit'} Constructing vaccination projections
2022/12/01 11:39:36 INFO|------------------------|{'fit_batch': '34', 'run_type': 'fit'} Retrieving hospitalizations data
2022/12/01 11:39:38 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'} Prepping Model


### Run the model

In [52]:
# now create the reports for the model, indexing through all the values of TC we want to simulate
logging.info('Projecting')
for tc in [0.7955, 0.7972, 0.8424]:
    logging.info('')
    #model.update_tc({model.date_to_t("2022-11-29"): {'co': tc}}, replace = False)
    model.update_tc({model.date_to_t(today): {'co': tc}}, replace = False)
    model.tags.update({'tc': tc})
    model.solve_seir()
    do_create_report(model, outdir=outdir, prep_model=False, solve_model=False, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

    logging.info('Running reports')

2022/12/01 11:43:34 INFO|Projecting
2022/12/01 11:43:34 INFO|
2022/12/01 11:43:39 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/01 11:45:58 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7955}: Running forward sim
2022/12/01 11:45:59 INFO|Running reports
2022/12/01 11:45:59 INFO|
2022/12/01 11:46:05 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/01 11:48:26 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7972}: Running forward sim
2022/12/01 11:48:28 INFO|Running reports
2022/12/01 11:48:28 INFO|
2022/12/01 11:48:34 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/01 11:50:56 INFO|{'fit_batch': '34', 'run_t